# 通过继承 `nn.Block`类来构造自己定义的层

In [3]:
import mxnet as mx
from mxnet import gluon,nd
from mxnet.gluon import nn


In [15]:
class MLP(nn.Block):
    #声明带有模型参数的类，这里声明了两个全连接层
    def __init__(self,**kargs): #通过重写构造函数来实现自己的层初始化
        #首先调用父类的构造函数
        super(MLP,self).__init__(**kargs)
        self.hidden = nn.Dense(256,activation='relu')
        self.output = nn.Dense(10)
    def forward(self,x):
        return self.output(self.hidden(x))

## 以上的MLP类中无需定义反向传播函数，mxnet会通过自动求梯度，从而求得反向传播所需的backward()

In [16]:
x = nd.random.uniform(shape=(2,20),ctx=mx.gpu())

In [17]:
net = MLP()

In [18]:
net.initialize(ctx = mx.gpu())

In [19]:
net(x)


[[ 0.05659141  0.02628279  0.01008523 -0.06831209 -0.04368097  0.01709548
   0.06409521 -0.01832914 -0.03691746  0.04692763]
 [ 0.07557119  0.01819326  0.01077965 -0.06995347 -0.07495361  0.0251789
   0.03733245 -0.04021479 -0.02011195  0.03191841]]
<NDArray 2x10 @gpu(0)>

## Sequential类 继承自Block，它提供add函数来逐一添加串联的Block子类，
## 下面手动实现一个Sequaltial类

In [20]:
class MySequential(nn.Block):
    
    def __init__(self,**kargs):
        super(MySequential,self).__init__(**kargs)
    #定义add函数
    def add(self,block):
        # block 是⼀个 Block ⼦类实例，假设它有⼀个独⼀⽆⼆的名字。我们将它保存在 Block
        # 类的成员变量 _children ⾥，其类型是 OrderedDict。当 MySequential 实例调⽤
        # initialize 函数时，系统会⾃动对 _children ⾥所有成员初始化。
        self._children[block.name] = block
    def forward(self,x):
        for block in self._children.values():
            x = block(x)
        return x

In [22]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize(ctx=mx.gpu())
net(x)


[[ 0.00423687 -0.01232926  0.04163517 -0.03118172  0.07519768 -0.01685334
  -0.00322678 -0.00201064  0.05564328  0.00303039]
 [-0.02585907  0.02349075  0.0347481  -0.02321478  0.088716   -0.0354879
  -0.02668689 -0.00363044  0.07213295 -0.00650949]]
<NDArray 2x10 @gpu(0)>

## 通过继承Block子类，可以定义一些复杂的模型，比如不被迭代的参数，以及实现控制流等

In [28]:
class FancyMLP(nn.Block):
    def __init__(self,**kargs):
        super(FancyMLP,self).__init__(**kargs)
        self.rand_weight = self.params.get_constant(
        'rand_weight',nd.random.uniform(shape=(20,20)))
        self.dense = nn.Dense(20,activation='relu')
        
    def forward(self,x):
        x = self.dense(x)
        #创建使用的常数参数，以及 NDarray 的relu 和dot函数
        x = nd.relu(nd.dot(x,self.rand_weight.data())+1)
        #重新使用层
        x = self.dense(x)
        
        while x.norm().asscalar() >1:
            x/=2
        if x.norm().asscalar() <0.8:
            x*=10
        return x.sum()
        

In [29]:
net = FancyMLP()
net.initialize(ctx= mx.gpu())
net(x)


[20.06795]
<NDArray 1 @gpu(0)>

## 嵌套调用

In [30]:
class NestMLP(nn.Block):
    def __init__(self,**kargs):
        super(NestMLP,self).__init__(**kargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(10,activation='relu'),
                    nn.Dense(20,activation='relu'))
        self.dense = nn.Dense(16,activation='relu')
    def forward(self,x):
        return self.dense(self.net(x))


In [31]:
net = nn.Sequential()

In [ ]:
net.add(NestMLP(),
        nn.Dense(10,activation='relu'),
        FancyMLP()
        )
net.initialize(ctx = mx.gou)